**Allocate Mem**

In [1]:
from tf_alloc import allocate as talloc
talloc(gpu=0, percentage=0.75)

2022-03-06 21:26:16.226717: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-06 21:26:16.235110: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-06 21:26:16.235913: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


## Example Model

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time
import numpy as np
import cv2

model_export_path="./part6/64x3-CNN.model"
image_for_prediction = "doggo.jpg"

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0
y = np.array(y)

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="part6/part6-logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.3,
                      callbacks=[tensorboard])

model.save(model_export_path)

3-conv-64-nodes-0-dense-1646634376


2022-03-06 21:26:16.475923: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-06 21:26:16.476697: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-06 21:26:16.477122: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-06 21:26:16.477399: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Epoch 1/10


2022-03-06 21:26:17.919934: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8302
2022-03-06 21:26:18.370765: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


546/546 [==============================] - 4s 5ms/step - loss: 0.6428 - accuracy: 0.6200 - val_loss: 0.5989 - val_accuracy: 0.6730
Epoch 2/10
546/546 [==============================] - 3s 5ms/step - loss: 0.5521 - accuracy: 0.7222 - val_loss: 0.5247 - val_accuracy: 0.7441
Epoch 3/10
546/546 [==============================] - 3s 5ms/step - loss: 0.4957 - accuracy: 0.7630 - val_loss: 0.5234 - val_accuracy: 0.7455
Epoch 4/10
546/546 [==============================] - 3s 5ms/step - loss: 0.4520 - accuracy: 0.7871 - val_loss: 0.4870 - val_accuracy: 0.7626
Epoch 5/10
546/546 [==============================] - 3s 5ms/step - loss: 0.4186 - accuracy: 0.8093 - val_loss: 0.4615 - val_accuracy: 0.7834
Epoch 6/10
546/546 [==============================] - 3s 5ms/step - loss: 0.3935 - accuracy: 0.8208 - val_loss: 0.5161 - val_accuracy: 0.7508
Epoch 7/10
546/546 [==============================] - 3s 5ms/step - loss: 0.3627 - accuracy: 0.8380 - val_loss: 0.4355 - val_accuracy: 0.7992
Epoch 8/10
546/54

2022-03-06 21:26:45.895297: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./part6/64x3-CNN.model/assets


### Test Model

In [3]:
import cv2
import tensorflow as tf

CATEGORIES = ["Dog", "Cat"]  # will use this to convert prediction num to string value
IMG_SIZE = 50  # 50 in txt-based

def prepare(filepath):
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # return the image with shaping that TF wants.

# Load Model
model = tf.keras.models.load_model(model_export_path)

# Make a Prediction
prediction = model.predict([prepare('doggo_ex.jpg')])
print(CATEGORIES[int(prediction[0][0])])

Cat
